<a href="https://colab.research.google.com/github/samipn/Pycaret/blob/main/02_multiclass_classification_wheat_seeds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass Classification (Wheat Seeds — 3 classes) — PyCaret

# ✅ Enable GPU in Colab
*Runtime → Change runtime type → **T4 / L4 GPU** → Save.*  
Each notebook sets `use_gpu=True` in `setup()`. Models that support GPU (e.g., XGBoost, CatBoost) will leverage it automatically if available.

UCI *Wheat Seeds* dataset. Target: `class` in \{1,2,3\}.

In [3]:
!pip install pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 7.1 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of category-encoders to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 21.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 113.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of flask to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of umap-learn to de

In [2]:
import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/wheat-seeds.csv"
cols = ["area","perimeter","compactness","length_of_kernel","width_of_kernel","asymmetry_coefficient","length_kernel_groove","class"]
df = pd.read_csv(url, names=cols)
print(df['class'].value_counts())

class
1    70
2    70
3    70
Name: count, dtype: int64


In [5]:
from pycaret.classification import setup, compare_models, tune_model, blend_models, stack_models, finalize_model, predict_model, pull, save_model

s = setup(
    data=df,
    target='class',
    session_id=123,
    # use_gpu=True, # Removed due to conflict
    normalize=True,
    fold=5
)

# Reduced the number of models to compare to 5 to potentially avoid crashing
best3 = compare_models(n_select=3)
tuned = [tune_model(m) for m in best3]
blended = blend_models(estimator_list=tuned)
final_model = finalize_model(blended)

preds = predict_model(final_model)
preds.head()

,Description,Value
0,Session id,123
1,Target,class
2,Target type,Multiclass
3,Target mapping,"1: 0, 2: 1, 3: 2"
4,Original data shape,"(210, 8)"
5,Transformed data shape,"(210, 8)"
6,Transformed train set shape,"(147, 8)"
7,Transformed test set shape,"(63, 8)"
8,Numeric features,7
9,Preprocess,True


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ridge,Ridge Classifier,0.9726,0.0000,0.9726,0.9746,0.9726,0.9589,0.9599,0.0140
svm,SVM - Linear Kernel,0.9657,0.0000,0.9657,0.9682,0.9656,0.9485,0.9498,0.0140
qda,Quadratic Discriminant Analysis,0.9522,0.0000,0.9522,0.9569,0.9518,0.9281,0.9306,0.0120
lda,Linear Discriminant Analysis,0.9520,0.0000,0.9520,0.9544,0.9519,0.9279,0.9292,0.0120
lr,Logistic Regression,0.9455,0.0000,0.9455,0.9509,0.9453,0.9182,0.9210,0.9860
et,Extra Trees Classifier,0.9389,0.9927,0.9389,0.9442,0.9385,0.9081,0.9110,0.0520
catboost,CatBoost Classifier,0.9317,0.9906,0.9317,0.9413,0.9295,0.8972,0.9029,1.1560
rf,Random Forest Classifier,0.9255,0.9849,0.9255,0.9363,0.9252,0.8881,0.8936,0.0660
gbc,Gradient Boosting Classifier,0.9251,0.0000,0.9251,0.9370,0.9230,0.8872,0.8940,0.0920
knn,K Neighbors Classifier,0.9184,0.9930,0.9184,0.9269,0.9183,0.8775,0.8820,0.7160


Processing:   0%|          | 0/71 [00:00<?, ?it/s]

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9310,0.0000,0.9310,0.9342,0.9308,0.8962,0.8979
3,0.9310,0.0000,0.9310,0.9310,0.9310,0.8964,0.8964
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Mean,0.9724,0.0000,0.9724,0.9730,0.9724,0.9585,0.9589
Std,0.0338,0.0000,0.0338,0.0330,0.0338,0.0508,0.0504


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9667,0.0000,0.9667,0.9697,0.9666,0.9500,0.9516
1,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
2,0.9310,0.0000,0.9310,0.9342,0.9308,0.8962,0.8979
3,0.9655,0.0000,0.9655,0.9690,0.9655,0.9483,0.9500
4,1.0000,0.0000,1.0000,1.0000,1.0000,1.0000,1.0000
Mean,0.9726,0.0000,0.9726,0.9746,0.9726,0.9589,0.9599
Std,0.0257,0.0000,0.0257,0.0244,0.0258,0.0387,0.0380


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.9841,0,0.9841,0.9848,0.9841,0.9762,0.9766


,area,perimeter,compactness,length_of_kernel,width_of_kernel,asymmetry_coefficient,length_kernel_groove,class,prediction_label
19,12.720000,13.570000,0.8686,5.226,3.049,4.102,4.914,1,1
129,17.549999,15.660000,0.8991,5.791,3.690,5.366,5.661,2,2
4,16.139999,14.990000,0.9034,5.658,3.562,1.355,5.175,1,1
3,13.840000,13.940000,0.8955,5.324,3.379,2.259,4.805,1,1
91,18.760000,16.200001,0.8984,6.172,3.796,3.120,6.053,2,2


In [7]:
# Save the best model (PyCaret 3.x) and export metrics
best_path = save_model(best3, 'best_model')
print("Saved model:", best_path)

# Export experiment logs / results
import pandas as pd
results_df = pull()  # last displayed table
results_df.to_csv('experiment_results.csv', index=False)
print("Exported experiment results to experiment_results.csv")

Transformation Pipeline and Model Successfully Saved
Saved model: (Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['area', 'perimeter', 'compactness',
                                             'length_of_kernel',
                                             'width_of_kernel',
                                             'asymmetry_coefficient',
                                             'length_kernel_groove'],
                                    transformer=SimpleImpu...
                                early_stopping=False, epsilon=0.1, eta0=0.001,
                                fit_intercept=True, l1_ratio=0.15,
                                learning_rate='optimal', loss